In [15]:
from glob import glob
import pandas as pd
import os
import nbformat

In [2]:
book2 = glob("../../notebooks/book2/*/*.ipynb")
len(book2), book2[0]

(183, '../../notebooks/book2/18/bnn_mnist_sgld.ipynb')

In [3]:
suffix = ["tfp", "blackjax", "jax", "pyro", "numpyro", "pymc3", "pymc", "sklearn", "flax"]
suffix = ["_" + suff for suff in suffix]  # add underscore

check_suffix = lambda nb_name: any([suff in nb_name for suff in suffix])
nb_suffix = list(filter(lambda x: check_suffix(x), book2))
len(nb_suffix), nb_suffix

(23,
 ['../../notebooks/book2/09/linreg_bayes_svi_hmc_pyro.ipynb',
  '../../notebooks/book2/09/svi_gmm_demo_2d_tfp.ipynb',
  '../../notebooks/book2/09/vb_gmm_tfp.ipynb',
  '../../notebooks/book2/03/linreg_divorce_ppc_numpyro.ipynb',
  '../../notebooks/book2/03/lkj_numpyro.ipynb',
  '../../notebooks/book2/03/hbayes_binom_rats_pymc3.ipynb',
  '../../notebooks/book2/03/schools8_pymc3.ipynb',
  '../../notebooks/book2/04/gibbs_demo_potts_jax.ipynb',
  '../../notebooks/book2/16/bnn_hierarchical_blackjax.ipynb',
  '../../notebooks/book2/11/funnel_numpyro.ipynb',
  '../../notebooks/book2/11/mcmc_traceplots_unigauss_numpyro.ipynb',
  '../../notebooks/book2/17/gpc_demo_2d_sklearn.ipynb',
  '../../notebooks/book2/15/activation_fun_deriv_jax.ipynb',
  '../../notebooks/book2/15/lecun1989_flax.ipynb',
  '../../notebooks/book2/14/linreg_hierarchical_non_centered_blackjax.ipynb',
  '../../notebooks/book2/14/linreg_height_weight_numpyro.ipynb',
  '../../notebooks/book2/14/logreg_iris_bayes_2d_pymc3.ipy

In [4]:
def replace_to_without_suffix(nb_path, do_replace):
    old_path = nb_path
    for suff in suffix:
        nb_path = nb_path.replace(suff, "")

    if do_replace:
        os.rename(old_path, nb_path)
    return (old_path, nb_path)

In [5]:
nb_pairs = list(map(replace_to_without_suffix, nb_suffix, [1] * len(suffix)))
nb_pairs

[('../../notebooks/book2/09/linreg_bayes_svi_hmc_pyro.ipynb',
  '../../notebooks/book2/09/linreg_bayes_svi_hmc.ipynb'),
 ('../../notebooks/book2/09/svi_gmm_demo_2d_tfp.ipynb',
  '../../notebooks/book2/09/svi_gmm_demo_2d.ipynb'),
 ('../../notebooks/book2/09/vb_gmm_tfp.ipynb',
  '../../notebooks/book2/09/vb_gmm.ipynb'),
 ('../../notebooks/book2/03/linreg_divorce_ppc_numpyro.ipynb',
  '../../notebooks/book2/03/linreg_divorce_ppc.ipynb'),
 ('../../notebooks/book2/03/lkj_numpyro.ipynb',
  '../../notebooks/book2/03/lkj.ipynb'),
 ('../../notebooks/book2/03/hbayes_binom_rats_pymc3.ipynb',
  '../../notebooks/book2/03/hbayes_binom_rats.ipynb'),
 ('../../notebooks/book2/03/schools8_pymc3.ipynb',
  '../../notebooks/book2/03/schools8.ipynb'),
 ('../../notebooks/book2/04/gibbs_demo_potts_jax.ipynb',
  '../../notebooks/book2/04/gibbs_demo_potts.ipynb'),
 ('../../notebooks/book2/16/bnn_hierarchical_blackjax.ipynb',
  '../../notebooks/book2/16/bnn_hierarchical.ipynb')]

## Update latex source code

In [6]:
tex_files = glob("../../../bookv2/book2/*/*.tex")
tex_files[5], len(tex_files)

('../../../bookv2/book2/comp/comp-methods.tex', 384)

In [7]:
def make_latex_nb(nb_path):
    """
    foo_jax -> foo\_jax
    """
    nb = nb_path.split("/")[-1]
    nb = nb.replace("_", "\_").replace(".ipynb", "")
    return nb


def replace_keyword_in_tex(tex_file, old_nb, new_nb):

    with open(tex_file, "r") as fp:
        content = fp.read()

    old = make_latex_nb(old_nb)  # foo_jax -> foo\_jax
    new = make_latex_nb(new_nb)

    if old in content:
        print(f"In {tex_file} - {old} -> {new}")
    content = content.replace(old, new)

    with open(tex_file, "w") as fp:
        fp.write(content)

In [73]:
replace_keyword_in_tex(tex_files[5], "advi_beta_binom_jax.ipynb", "advi_beta_binom.ipynb")

In ../../../bookv2/book2/comp/comp-methods.tex - advi_beta_binom_jax.ipynb -> advi_beta_binom.ipynb


In [10]:
for pair in nb_pairs:
    old, new = pair
    # print(old, new)
    list(map(replace_keyword_in_tex, tex_files, [old] * len(tex_files), [new] * len(tex_files)))

## Update in notebooks 

In [11]:
def apply_fun_to_notebook(notebook, fun, old_nb_path, new_nb_path):
    """
    fun should take one argument: code
    """
    nb = nbformat.read(notebook, as_version=4)
    for cell in nb.cells:
        code = cell["source"]
        updated_code = fun(code, old_nb_path, new_nb_path)
        if updated_code != code:
            cell["source"] = updated_code
            nbformat.write(nb, notebook)


def replace_keyword(code, old_nb_path, new_nb_path):
    old_nb = old_nb_path.split("/")[-1]
    new_nb = new_nb_path.split("/")[-1]
    code = code.replace(old_nb, new_nb)
    return code

In [19]:
book2 = glob("../../notebooks/book2/*/*.ipynb")
for pair in nb_pairs:
    list(
        map(
            apply_fun_to_notebook, book2, [replace_keyword] * len(book2), [pair[0]] * len(book2), [pair[1]] * len(book2)
        )
    )